In [3]:
!pip install -r requirements.txt
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

Defaulting to user installation because normal site-packages is not writeable


In [4]:
support_tickets = [
    {"text": "My account login is not working. I've tried resetting my password twice."},
    {"text": "The app crashes every time I try to upload a photo."},
    {"text": "I was charged twice for my last subscription payment."},
    {"text": "I can't find the option to change my profile picture."},
    {"text": "The video playback is very laggy on my device."}
]
knowledge_base = [
    "Category 1 -Login Issues -Login issues often occur due to incorrect passwords or account lockouts.",
    "Category 2 -App Functionality -App crashes can be caused by outdated software or device incompatibility.",
    "Category 3 -Billing -Billing discrepancies may result from processing errors or duplicate transactions.",
    "Category 4 -Account Management -Account management includes tasks such as changing profile information, linking social media accounts, and managing privacy settings.",
    "Category 5 -Performance Issues -Performance issues can be related to device specifications, network connectivity, or app optimization."
]

In [ ]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [6]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [7]:
knowledge_base_embeddings = np.array([get_embedding(cat) for cat in knowledge_base])


In [8]:
def retrieve_most_relevant(query_embedding, top_k=1):
    similarities = cosine_similarity([query_embedding], knowledge_base_embeddings)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [knowledge_base[i] for i in top_indices]

In [9]:
def classify_ticket(ticket_text):
    ticket_embedding = get_embedding(ticket_text)
    relevant_category = retrieve_most_relevant(ticket_embedding)[0]
    classification = relevant_category.split(" -")[0]
    return classification

In [10]:
for ticket in support_tickets:
    classification = classify_ticket(ticket['text'])
    print(f"Ticket: {ticket['text']}\nClassification: {classification}\n")

Ticket: My account login is not working. I've tried resetting my password twice.
Classification: Category 1

Ticket: The app crashes every time I try to upload a photo.
Classification: Category 2

Ticket: I was charged twice for my last subscription payment.
Classification: Category 3

Ticket: I can't find the option to change my profile picture.
Classification: Category 4

Ticket: The video playback is very laggy on my device.
Classification: Category 5



In [11]:
def evaluate_classifier(test_tickets):
    correct = 0
    total = len(test_tickets)

    for ticket in test_tickets:
        true_label = ticket["label"]
        predicted_label = classify_ticket(ticket["text"])
        if predicted_label == true_label:
            correct += 1

    accuracy = correct / total
    print(f"Accuracy: {accuracy:.2f}")

In [12]:
test_tickets = [
    {"text": "I can't log in to my account. It says my password is incorrect.", "label": "Category 1"},
    {"text": "The app keeps crashing when I open it on my iPhone.", "label": "Category 2"},
    {"text": "I was charged $19.99 twice this month for my subscription.", "label": "Category 3"},
    {"text": "How do I change my profile picture? I can't find the option.", "label": "Category 4"},
    {"text": "Videos are buffering constantly and the quality is poor.", "label": "Category 5"},
    {"text": "I forgot my username. How can I recover it?", "label": "Category 1"},
    {"text": "The app freezes when I try to send a message.", "label": "Category 2"},
    {"text": "I cancelled my subscription but I'm still being charged.", "label": "Category 3"},
    {"text": "Is there a way to make my account private?", "label": "Category 4"},
    {"text": "The app is using too much battery on my device.", "label": "Category 5"},
    {"text": "I'm getting an error message when trying to reset my password.", "label": "Category 1"},
    {"text": "The search function in the app isn't working properly.", "label": "Category 2"},
    {"text": "Can I get a refund for an accidental purchase?", "label": "Category 3"},
    {"text": "How do I link my Facebook account to my profile?", "label": "Category 4"},
    {"text": "The app is very slow to load on my Android phone.", "label": "Category 5"},
    {"text": "I'm locked out of my account after too many login attempts.", "label": "Category 1"},
    {"text": "The notification sounds aren't working in the app.", "label": "Category 2"},
    {"text": "I don't recognize a charge on my account statement.", "label": "Category 3"},
    {"text": "How can I change my email address associated with my account?", "label": "Category 4"},
    {"text": "The live streaming feature is lagging and cutting out frequently.", "label": "Category 5"}
]


evaluate_classifier(test_tickets)

Accuracy: 0.95


In [13]:
# Test the classifier with a specific example
test_ticket = "My account login is not working. I've tried resetting my password twice."
classification = classify_ticket(test_ticket)
print(f"Ticket: {test_ticket}\nClassification: {classification}\n")

# Additional testing with other sample tickets
for ticket in support_tickets:
    classification = classify_ticket(ticket['text'])
    print(f"Ticket: {ticket['text']}\nClassification: {classification}\n")


Ticket: My account login is not working. I've tried resetting my password twice.
Classification: Category 1

Ticket: My account login is not working. I've tried resetting my password twice.
Classification: Category 1

Ticket: The app crashes every time I try to upload a photo.
Classification: Category 2

Ticket: I was charged twice for my last subscription payment.
Classification: Category 3

Ticket: I can't find the option to change my profile picture.
Classification: Category 4

Ticket: The video playback is very laggy on my device.
Classification: Category 5



In [14]:
# Define the test tickets
test_tickets = [
    {"text": "I can't log in to my account. It says my password is incorrect.", "label": "Category 1"},
    {"text": "The app keeps crashing when I open it on my iPhone.", "label": "Category 2"},
    {"text": "I was charged $19.99 twice this month for my subscription.", "label": "Category 3"},
    {"text": "How do I change my profile picture? I can't find the option.", "label": "Category 4"},
    {"text": "Videos are buffering constantly and the quality is poor.", "label": "Category 5"},
    {"text": "I forgot my username. How can I recover it?", "label": "Category 1"},
    {"text": "The app freezes when I try to send a message.", "label": "Category 2"},
    {"text": "I cancelled my subscription but I'm still being charged.", "label": "Category 3"},
    {"text": "Is there a way to make my account private?", "label": "Category 4"},
    {"text": "The app is using too much battery on my device.", "label": "Category 5"},
    {"text": "I'm getting an error message when trying to reset my password.", "label": "Category 1"},
    {"text": "The search function in the app isn't working properly.", "label": "Category 2"},
    {"text": "Can I get a refund for an accidental purchase?", "label": "Category 3"},
    {"text": "How do I link my Facebook account to my profile?", "label": "Category 4"},
    {"text": "The app is very slow to load on my Android phone.", "label": "Category 5"},
    {"text": "I'm locked out of my account after too many login attempts.", "label": "Category 1"},
    {"text": "The notification sounds aren't working in the app.", "label": "Category 2"},
    {"text": "I don't recognize a charge on my account statement.", "label": "Category 3"},
    {"text": "How can I change my email address associated with my account?", "label": "Category 4"},
    {"text": "The live streaming feature is lagging and cutting out frequently.", "label": "Category 5"}
]

# Run the classifier on each ticket
for ticket in test_tickets:
    classification = classify_ticket(ticket['text'])
    print(f"Ticket: {ticket['text']}\nExpected Label: {ticket['label']}\nPredicted Classification: {classification}\n")


Ticket: I can't log in to my account. It says my password is incorrect.
Expected Label: Category 1
Predicted Classification: Category 1

Ticket: The app keeps crashing when I open it on my iPhone.
Expected Label: Category 2
Predicted Classification: Category 2

Ticket: I was charged $19.99 twice this month for my subscription.
Expected Label: Category 3
Predicted Classification: Category 3

Ticket: How do I change my profile picture? I can't find the option.
Expected Label: Category 4
Predicted Classification: Category 4

Ticket: Videos are buffering constantly and the quality is poor.
Expected Label: Category 5
Predicted Classification: Category 5

Ticket: I forgot my username. How can I recover it?
Expected Label: Category 1
Predicted Classification: Category 1

Ticket: The app freezes when I try to send a message.
Expected Label: Category 2
Predicted Classification: Category 2

Ticket: I cancelled my subscription but I'm still being charged.
Expected Label: Category 3
Predicted Clas